# scratch work

In [1]:
import astropy.units as u
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.time import Time

from jorbit.mpchecker import (
    load_mpcorb,
    mpchecker,
    nearest_asteroid,
    nearest_asteroid_helper,
)

In [2]:
precomputed = nearest_asteroid_helper(coordinate=SkyCoord(ra=0*u.deg, dec=0*u.deg), times=Time("2025-02-01") + np.arange(0, 3, 1/24/4)*u.day)

In [3]:
dists = nearest_asteroid(
    coordinate=SkyCoord(ra=0*u.deg, dec=0*u.deg),
    times=Time("2025-02-01") + np.arange(0, 3, 1/24/4)*u.day,
    precomputed=precomputed
)



original number of possible asteroids: 2
number of asteroids within 30 degrees: 271351
going to be looping through 192 offsets
original number of possible asteroids: 2
number of asteroids within 30 degrees: 272663
going to be looping through 96 offsets


In [5]:
mpchecker(
    coordinate=SkyCoord(ra=0*u.deg, dec=0*u.deg),
    time=Time("2025-02-01"),
    radius=5*u.arcmin,
)


name,separation,ra,dec
,arcsec,deg,deg
str7,float64,float64,float64
270258,104.1808207360553,359.97292932446953,-0.01022990808315794
51910,143.31642575663903,0.018295392777435124,-0.035357094930712325
137143,246.1114123633815,359.93371498406174,-0.01673235585937548
254903,270.33085614398374,359.9886589111433,-0.07423054556318036
739168,274.188362870427,359.96852902476695,-0.06935738533299034


In [2]:
mpcorb = load_mpcorb()

In [14]:
"270258" in mpcorb["Packed designation"].to_list()

False

In [9]:
mpcorb

Packed designation,H,G,Epoch,M,Peri,Node,Incl.,e,n,a,U,Reference,#Obs,#Opp,Arc,rms,Coarse Perts,Precise Perts,Computer,Flags,Unpacked Name,last obs
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""00001""","""3.34""","""0.15""","""K24AH""","""145.84905""","""73.28579""","""80.25414""","""10.58790""","""0.0791840""","""0.21418047""","""2.7666197""","""0""","""E2024-V47""","""7330""","""125""","""1801-2024""","""0.80""","""M-v""","""30k""","""MPCLINUX""","""4000""","""(1) Ceres""","""20241101"""
"""00002""","""4.11""","""0.15""","""K24AH""","""126.06756""","""310.89226""","""172.90614""","""34.92186""","""0.2304384""","""0.21374870""","""2.7703442""","""0""","""E2024-V16""","""8964""","""123""","""1804-2024""","""0.75""","""M-c""","""28k""","""Pan""","""4000""","""(2) Pallas""","""20240927"""
"""00003""","""5.18""","""0.15""","""K24AH""","""127.32529""","""247.81975""","""169.83829""","""12.98815""","""0.2561092""","""0.22588717""","""2.6701869""","""0""","""E2024-V16""","""7523""","""117""","""1804-2024""","""0.83""","""M-v""","""3Ek""","""Pan""","""4000""","""(3) Juno""","""20240607"""
"""00004""","""3.25""","""0.15""","""K24AH""","""278.02316""","""151.67629""","""103.70474""","""7.14398""","""0.0900011""","""0.27169443""","""2.3609252""","""0""","""E2024-V16""","""7647""","""111""","""1821-2024""","""0.82""","""M-p""","""18k""","""Pan""","""4000""","""(4) Vesta""","""20240306"""
"""00005""","""6.99""","""0.15""","""K243V""","""350.98291""","""359.23648""","""141.46063""","""5.35914""","""0.1872507""","""0.23840266""","""2.5758979""","""0""","""MPO840811""","""3344""","""87""","""1845-2024""","""0.71""","""M-v""","""3Ek""","""MPCLINUX""","""4000""","""(5) Astraea""","""20240603"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""K21C10K""","""20.8""","""0.15""","""K212F""","""269.71523""","""243.15772""","""351.93463""","""25.15937""","""0.0679376""","""0.50917447""","""1.5531914""",null,"""E2021-D26""","""10""","""1""","""5 days""","""0.24""",null,null,"""Bell""","""2005""","""2021 CK10""","""20210217"""
"""K21C10S""","""20.6""","""0.15""","""K212F""","""164.66648""","""223.65372""","""119.89392""","""29.38693""","""0.0526282""","""0.44236564""","""1.7058799""",null,"""E2021-DA1""","""9""","""1""","""11 days""","""0.07""",null,null,"""Bell""","""2005""","""2021 CS10""","""20210223"""
"""K21C16A""","""20.3""","""0.15""","""K212F""","""177.55080""","""227.21125""","""95.72419""","""24.84879""","""0.0475499""","""0.38977199""","""1.8560755""",null,"""E2021-E98""","""9""","""1""","""5 days""","""0.12""",null,null,"""Bell""","""2006""","""2021 CA16""","""20210217"""


In [ ]:
def mpc_code_to_number(code):

    # if it's a numbered object, it could be written 3 forms:

    # low numbered objects are just numbers
    if code.isdigit():
        return code

    # medium-numbered objects are a letter followed by 4 digits
    def letter_to_number(char):
        if char.isupper():
            return ord(char) - ord("A") + 10
        else:
            return ord(char) - ord("a") + 36

    if code[0].isalpha() and code[1:].isdigit():
        prefix_value = letter_to_number(code[0])
        num = (prefix_value * 10000) + int(code[1:])
        return str(num)

    # high-numbered objects are a tilde followed by a base-62 number
    def base62_to_decimal(char):
        if char.isdigit():
            return int(char)
        elif char.isupper():
            return ord(char) - ord("A") + 10
        else:
            return ord(char) - ord("a") + 36

    if code.startswith("~"):
        # Convert each character to its decimal value and calculate total
        total = 0
        for position, char in enumerate(reversed(code[1:])):
            decimal_value = base62_to_decimal(char)
            total += decimal_value * (62**position)
        num = total + 620000
        return str(num)